# OLAP style analysis

We have a fact table of laptops that gives the product price in Amazon and Walmart. Some dimensions in this table are brand, operating system, processor (cpu) manufacturer etc.

We now perform some olap-style analysis on the table.

In [1]:
import pandas as pd
import mlxtend
from collections import defaultdict
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import sys

df = pd.read_csv('./data/merged_clean_numeric_v2.csv')
df.insert(0, 'id', range(0, len(df)))

print('\tamazon_price\twalmart_price')
print ('Mean\t' +  str(df['amazon_price'].mean()) + '\t' + str(df['walmart_price'].mean()))

	amazon_price	walmart_price
Mean	1385.5805966930266	1571.5141073568175


**Observation 1: In general, prices are expensive in Walmart than in Amazon.**

In [2]:
print ('\nDrilling down by brand:')
print (df.groupby(['brand'])['amazon_price', 'walmart_price'].mean())


Drilling down by brand:
           amazon_price  walmart_price
brand                                 
AORUS       2299.000000    2199.000000
Acer         487.122361     509.250694
Alienware   1785.146316    1714.692632
Aorus       2199.000000    2099.000000
Apple       1118.584123     729.805614
Asus         994.911304     864.117681
Bit          214.990000     179.000000
CTL          292.970000     264.810000
Dell         740.719229     692.582207
HP           928.033926    1066.269453
Huawei       689.280000     662.200000
LG          1247.030500     894.477000
Lenovo      1672.351025    2018.848058
MSI         1363.211951    1352.693659
Microsoft   2542.027500    2071.745000
OMEN        3599.000000    3563.950000
Panasonic   1224.791250     728.715000
Prostar     1318.549763    1217.438270
Razer       1604.247500    1790.990000
SLIDE        224.990000     224.990000
Samsung      972.627778     780.921111
Toshiba     1322.960000    1319.400000
VIZIO        979.990000     364.495000


**Observation 2: But if we drill down on the prices, using a particular brand, then we find that for a few brands like Apple, Alienware, Asus, Dell, etc Amazon is pricier.**

In [3]:
df.insert(len(df.columns), 'average_price', (df['amazon_price'] + df['walmart_price'])/2.0)
temp = (df['average_price'])
temp = list(map(int, temp))
temp = [str(int(i/500)*500) + '-' + str(int(i/500)*500 + 500) for i in temp]
df.insert(len(df.columns), 'average_price_range', temp)

In [4]:
filtered = df.groupby(['brand']).filter(lambda x: len(x) >= 100)
filtered.groupby(['brand', 'average_price_range'])['id'].count()

brand    average_price_range
Acer     0-500                    87
         1000-1500                 8
         1500-2000                 2
         500-1000                 47
Apple    0-500                     1
         1000-1500                27
         1500-2000                 6
         2000-2500                 2
         500-1000                 78
Dell     0-500                    71
         1000-1500                52
         1500-2000                 8
         2000-2500                 2
         500-1000                243
HP       0-500                    89
         1000-1500               248
         1500-2000                46
         2000-2500                 4
         500-1000                125
Lenovo   0-500                    39
         1000-1500                66
         1500-2000              1592
         2000-2500               478
         2500-3000               133
         3000-3500                 2
         500-1000                100
Prostar  



**Observation 3: If we look at the average price range, Lenevo has a lot of products whose price > 1500$**


In [5]:
print (df.groupby(['brand'])['id'].count())

brand
AORUS           1
Acer          144
Alienware      19
Aorus           1
Apple         114
Asus           69
Bit             2
CTL             1
Dell          376
HP            512
Huawei          1
LG             20
Lenovo       2410
MSI            41
Microsoft       4
OMEN            2
Panasonic       8
Prostar       422
Razer           4
SLIDE           1
Samsung        18
Toshiba         1
VIZIO           2
Name: id, dtype: int64


**There a lot of Lenevo products in our database, so for further analysis let's consider only the lenevo brand.**

In [6]:
print (df[df['brand'] =="Lenovo"].groupby(['operating system'])['id'].count())

operating system
Chrome          26
WINDOWS 10       1
Windows 10    2163
Windows 7      215
Windows 8        5
Name: id, dtype: int64


**Observation 4: Most of the Lenevo products in our database have Windows 10 as their operating systems**